In [13]:
import numpy as np 
import pandas as pd 

import surprise

from surprise import SVD
from surprise import Reader, Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommendationsv4/train.csv
/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json
/kaggle/input/recommendationsv4/test.csv
/kaggle/input/recommendationsv4/sample_submission.csv


In [14]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
submission.head()

,Id,rating
0,0,0.282813
1,1,0.629758
2,2,0.916899
3,3,0.702790
4,4,0.043446


фиксируем random.seed

In [16]:
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

<bound method Random.seed of <random.Random object at 0x564fc3082470>>

Обнаружила 2 подхода: обучение на всем датасете и разбивка на обучающую и тестовую выборки.

обучение на всем датасете

In [17]:
#определяем модель, в нашем случае это будет SDV
svd = SVD()
# так как датасет у наш внешний, не из библиотеки surprise, то нам нужен ридер, в нем обозначаем масштаб
reader = Reader(rating_scale=(0,1))
#загружаем датасет и обозначаем нужные поля
data = Dataset.load_from_df(train[['userid','itemid', 'rating']], reader)
#получаем обучающую выборку
trainingSet = data.build_full_trainset()
#обучаем модель
svd.fit(trainingSet)
#получаем тестовую выборку
testSet = trainingSet.build_testset()
#предсказываем значения рейтинга на тестовой выборке
prediction = svd.test(testSet)
#считаем RMSE (не нашла в surprise готовых формул для roc_auc, precision и recall, есть фукнции, позже вернусь)
#и по этой метрике выберем модель
accuracy.rmse(prediction, verbose=True)

RMSE: 0.2640


0.26403486300703966

результат, полученный с помощью метода test, выглядит так

In [10]:
prediction

[Prediction(uid=102179, iid=37138, r_ui=1.0, est=0.8004788900307461, details={'was_impossible': False}),
 Prediction(uid=102179, iid=9359, r_ui=0.0, est=0.8647720496640908, details={'was_impossible': False}),
 Prediction(uid=102179, iid=37701, r_ui=1.0, est=0.842792700594155, details={'was_impossible': False}),
 Prediction(uid=102179, iid=37372, r_ui=1.0, est=0.8067187834356588, details={'was_impossible': False}),
 Prediction(uid=102179, iid=37320, r_ui=1.0, est=0.9269722282274866, details={'was_impossible': False}),
 Prediction(uid=102179, iid=37318, r_ui=1.0, est=0.8473486905490989, details={'was_impossible': False}),
 Prediction(uid=3625, iid=17322, r_ui=1.0, est=1, details={'was_impossible': False}),
 Prediction(uid=3625, iid=22230, r_ui=1.0, est=0.949354771762796, details={'was_impossible': False}),
 Prediction(uid=3625, iid=13526, r_ui=1.0, est=1, details={'was_impossible': False}),
 Prediction(uid=3625, iid=15633, r_ui=1.0, est=1, details={'was_impossible': False}),
 Prediction(

вот так можно сделать датафрейм в привычном виде

In [18]:
df = pd.DataFrame(prediction, columns=['uid', 'iid', 'rui', 'est', 'details'])

In [19]:
df.head()

,uid,iid,rui,est,details
0,102179,37138,1.0,0.862215,{'was_impossible': False}
1,102179,9359,0.0,0.760161,{'was_impossible': False}
2,102179,37701,1.0,0.986732,{'was_impossible': False}
3,102179,37372,1.0,0.666838,{'was_impossible': False}
4,102179,37320,1.0,0.863953,{'was_impossible': False}


разбивка на обучающую и тестовую выборки

In [21]:
#определяем модель, в нашем случае это будет SDV, назовем его по-другому
algo = SVD()
# так как датасет у наш внешний, не из библиотеки surprise, то нам нужен ридер, в нем обозначаем масштаб
reader = Reader(rating_scale=(0,1))
#загружаем датасет и обозначаем нужные поля
data = Dataset.load_from_df(train[['userid','itemid', 'rating']], reader)
#разбиваем на тестовую и обучающую выборки
trainset, testset = train_test_split(data, test_size=.25)
#Обучаем модель и предсказываем значения
algo.fit(trainset)
predictions = algo.test(testset)
#считаем RMSE
accuracy.rmse(predictions, verbose=True)

RMSE: 0.3456


0.3455507913064255

теперь посмотрим, как отработает метод predict

In [22]:
uid = str(196)  # выбираем юзера
iid = str(302)  # выбираем айтем

# предсказываем для конкретного юзера и айтема
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 0.85   {'was_impossible': False}


а как использовать predict для всех и сразу юзеров и айтемов?

в цикле ОЧЕНЬ долго, хотя возможно

In [ ]:
for uid in test.userid.values.tolist():
    for iid in test.itemid.values.tolist():
        pred = algo.predict(uid, iid,verbose=True)
        
df = pd.DataFrame(pred, columns=['uid', 'iid', 'rui', 'est', 'details'])

попробуем написать функцию

так как алгоритм оценивали по метрике rsme, ктр чем меньше, тем лучше, то выбираем 1й вариант, с обучением на всем датафрейме

In [32]:
#определяем функцию для предикта, передаем в нее юзера и айтем
def predict(row):
    pr = svd.predict(row['userid'], row['itemid'])
    df = pd.DataFrame([pr], columns=['uid', 'iid', 'rui', 'est', 'details'])
    return df.est

#добавляем в датасет тест новую колонку с предсказанными значениями
test['rating'] = test.apply(predict, axis = 1)

In [33]:
submission = test[['Id', 'rating']]
submission.to_csv('submission.csv', index=False)

результат на kaggle 0.72114, что хуже бейзлайна

вот еще способ, как с помощью метода тест делать предсказания

In [70]:
#создаем дамми колонку в тестовом датасете, так как она понадобится для шага 2
test.loc[:, 'rating'] = 0 
#используем load_from_df чтобы сконвертировать тестовый датасет в тот формат, ктр нужен для шага 3
test_processed = Dataset.load_from_df(test[['userid','itemid','rating']], reader) 
# используем train_test_split чтобы сконвертировать test_processed в список кортежей для шага 4 
NA, test = train_test_split(test_processed, test_size=1.0) 
#предсказываем, используя модель svd
predictions = svd.test(test)
#достаем значения рейтинга из колонки est, полученные на предыдущем шаге
est = [i.est for i in predictions] 

In [71]:
submission['rating'] = est

In [72]:
submission.to_csv('submission.csv', index=False)

результат на kaggle 0.69763, что хуже бейзлайна и хуже результата, полученного с помощью метода predict